In [ ]:
#Script created by Neha on 2022-10-04
#Script is designed to analyze Applebees Site traffic analysis on APP only
#Github repo: https://github.com/matt-walentosky-bt/applebees_user_pathing_analysis

In [1]:
#Import desired packages
import ga4_user_pathing
import pandas as pd
import os

In [2]:
#Parameters from Tyler Norris / Vanessa Schneider
# Globals:
COUNT_FIELD = 'counts' # DF fields references to aggregations value (users, sessions)
NODE_OPACITY = .9
LINK_OPACITY = .6
SEP_VAL = ' > '
KEY_PATH = '/Users/vanessaschneider/keys/dse-chase-user-pathing.json'

In [3]:
#Get working directory
os.getcwd()

'/Users/nehakhatri/Documents/GitHub/applebees_user_pathing_analysis'

In [4]:
#Read desired document
conversion_status = True

if conversion_status == True:
    df = pd.read_csv("APP_converters_user_pathing.csv")
else:
    df = pd.read_csv("APP_non_converters_user_pathing.csv")
    
#Note, we can only pull fraction of data from BQ into CSV format   

In [5]:
print(df.shape)
print("----")
print(df.head())

(16204, 3)
----
                                             journey  conversion_flg  counts
0  viewed_favourite_store > Menu > added_item > C...            True    1381
1               Menu > added_item > CheckoutActivity            True    1303
2         Menu > added_item > CheckoutViewController            True    1165
3             Quick_Reorder > CheckoutViewController            True    1025
4  viewed_favourite_store > added_item > Checkout...            True     753


In [6]:
df['journey_length'] = df.apply(lambda x: len(x['journey'].split(SEP_VAL)), axis=1)

In [ ]:
df.describe()

In [25]:
df.head()
df.shape

(16204, 14)

In [26]:
df.to_csv(('/Users/nehakhatri/Local_Python/Applebees/converters_journey_length.csv'))

In [ ]:
# Change min journey lenght and number of bins to further explore
df[df['journey_length'] > 0]['journey_length'].hist(bins=8)

In [ ]:
df[df['journey_length'] > 1]['counts'].hist(bins=30)

In [ ]:
df.head(25)

In [7]:
df = ga4_user_pathing.add_journey_end_string(df, journey_col='journey', conversion_column='Conversion', sep_val=' > ')
df = ga4_user_pathing.parse_steps(df, sep_val=' > ', max_steps=10)

In [8]:
df['journey'][0]

'viewed_favourite_store > Menu > added_item > CheckoutViewController > Journey End'

In [ ]:
#df_filtered.columns

In [ ]:
#df_filtered = ga4_user_pathing.remove_long_tail_jouneys(df, count_col='counts', min_path_count=10)

In [21]:
df_filtered = df[(df['journey_length'] >= 1) & (df['journey_length'] <= 6)]
df_filtered = df_filtered.sort_values('counts', ascending = False)
df_filtered = df_filtered.iloc[0:100,:]
print(df_filtered.shape)

(100, 14)


In [10]:
df_filtered = df_filtered.iloc[0:10,:]
print(df_filtered.shape)

(10, 14)


In [20]:
df_filtered.head()

,journey,conversion_flg,counts,journey_length,step_1,step_2,step_3,step_4,step_5,step_6,step_7,step_8,step_9,step_10
5,Menu > added_item > Menu > added_item > Checko...,True,753,5,Menu,added_item,Menu,added_item,CheckoutActivity,Journey End,NaN,NaN,NaN,NaN
6,viewed_favourite_store > Menu > added_item > M...,True,750,6,viewed_favourite_store,Menu,added_item,Menu,added_item,CheckoutViewController,Journey End,NaN,NaN,NaN
10,Menu > added_item > Menu > added_item > Checko...,True,557,5,Menu,added_item,Menu,added_item,CheckoutViewController,Journey End,NaN,NaN,NaN,NaN
15,viewed_favourite_store > Menu > added_item > A...,True,266,6,viewed_favourite_store,Menu,added_item,Appetizers,added_item,CheckoutViewController,Journey End,NaN,NaN,NaN
16,viewed_favourite_store > Appetizers > added_it...,True,250,6,viewed_favourite_store,Appetizers,added_item,Menu,added_item,CheckoutViewController,Journey End,NaN,NaN,NaN


In [16]:

df_filtered.to_csv('/Users/nehakhatri/Local_Python/Applebees/converters_top10_ge5_jl.csv')

In [11]:
df_filtered.columns

Index(['journey', 'conversion_flg', 'counts', 'journey_length', 'step_1',
       'step_2', 'step_3', 'step_4', 'step_5', 'step_6', 'step_7', 'step_8',
       'step_9', 'step_10'],
      dtype='object')

In [22]:
df_sankey, df_index = ga4_user_pathing.generate_sankey_inputs(df_filtered, COUNT_FIELD, .9)

/Users/nehakhatri/Documents/GitHub/applebees_user_pathing_analysis/ga4_user_pathing.py:253: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

/Users/nehakhatri/Documents/GitHub/applebees_user_pathing_analysis/ga4_user_pathing.py:261: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [ ]:
#df_filtered.head(15)
#df_filtered.to_csv('/Users/nehakhatri/Local_Python/Applebees/non_converters_top10.csv')
#df_filtered.to_csv('/Users/nehakhatri/Local_Python/Applebees/non_converters_top1_ge3_jl.csv')

In [13]:
df_sankey.head()

,source,target,counts,source_index,color,target_index
0,Appetizers,added_item,2071,4,"rgba(244, 14, 164, 0.9)",0
1,CheckoutActivity,Journey End,1186,6,"rgba(255, 177, 255, 0.9)",2
2,CheckoutViewController,Journey End,2485,3,"rgba(128, 68, 123, 0.9)",2
3,Menu,added_item,5271,1,"rgba(53, 41, 127, 0.9)",0
4,added_item,Appetizers,930,0,"rgba(142, 140, 153, 0.9)",4


In [23]:
ga4_user_pathing.display_sankey(df_sankey, df_index, count_field=COUNT_FIELD, link_opacity=.3)